In [ ]:
import numpy as np
import pandas as pd

# Parámetros del problema
Nurses = 10 # Número de enfermeras
Days = 30  # Días en un mes
Shifts = 3  # Turnos por día (mañana, tarde, noche)
Population_Size = 80
Generations = 10 # 10.000
Mutation_Probability = 0.01
Crossover_Probability = 1
Elitism_Size = 40

In [7]:
# Generar una matriz aleatoria de enfermeras para un cromosoma
def generate_random_chromosome(Nurses, Days, Shifts):
    # Este np.random.choice() genera un cromosoma aleatorio con 10 enfermeras
    # en cada turno de cada día. Cada enfermera se representa con un número
    chromosome = np.random.choice(Nurses, (Days, Shifts, 2))
    return chromosome

# Los cromosomas para este caso siempre son validos
def valid(chromosome):
    return True

# Inicializar población
def generateFirstPopulation(Population_Size, Nurses, Days, Shifts):
    population = []
    while len(population) < Population_Size:
        chromosome = generate_random_chromosome(Nurses, Days, Shifts)
        if valid(chromosome):
            population.append(chromosome)
    return population

# Función de aptitud (fitness)
def count_violations(chromosome):
    violations = 0
    
    # Regla 1: "Una enfermera por turno y día de trabajo."
    for day in range(Days):
        for shift in range(Shifts):
            nurses_in_shift = chromosome[day][shift]
            if len(nurses_in_shift) != len(set(nurses_in_shift)):
                violations += 1  # Penaliza si hay duplicados en el turno

    # Regla 2: "No más de dos turnos de mañana consecutivos."
    for nurse in range(Nurses):
        consecutive_morning_shifts = 0
        for day in range(Days):
            if nurse in chromosome[day][0]:  # Turno de mañana
                consecutive_morning_shifts += 1
                if consecutive_morning_shifts > 2:
                    violations += 1
            else:
                consecutive_morning_shifts = 0

    # Regla 3: "No más de dos turnos de tarde consecutivos."
    for nurse in range(Nurses):
        consecutive_afternoon_shifts = 0
        for day in range(Days):
            if nurse in chromosome[day][1]:  # Turno de tarde
                consecutive_afternoon_shifts += 1
                if consecutive_afternoon_shifts > 2:
                    violations += 1
            else:
                consecutive_afternoon_shifts = 0

    # Regla 4: "No más de dos turnos de noche consecutivos."
    for nurse in range(Nurses):
        consecutive_night_shifts = 0
        for day in range(Days):
            if nurse in chromosome[day][2]:  # Turno de noche
                consecutive_night_shifts += 1
                if consecutive_night_shifts > 2:
                    violations += 1
            else:
                consecutive_night_shifts = 0

    # Regla 5: "No más de dos días libres consecutivos."
    for nurse in range(Nurses):
        consecutive_days_off = 0
        for day in range(Days):
            if all(nurse not in chromosome[day][shift] for shift in range(Shifts)):
                consecutive_days_off += 1
                if consecutive_days_off > 2:
                    violations += 1
            else:
                consecutive_days_off = 0

    # Regla 6: "Turnos de noche según la porción predeterminada."
    # Asumimos que queremos que cada enfermera tenga aproximadamente el mismo número de turnos de noche.
    expected_night_shifts = Days * Shifts // Nurses
    for nurse in range(Nurses):
        night_shifts = sum(nurse in chromosome[day][2] for day in range(Days))
        if abs(night_shifts - expected_night_shifts) > 1:
            violations += 1

    # Regla 7: "Día libre después de dos turnos de noche consecutivos."
    for nurse in range(Nurses):
        for day in range(1, Days - 1):
            if (nurse in chromosome[day-1][2] and nurse in chromosome[day][2] and
                all(nurse not in chromosome[day+1][shift] for shift in range(Shifts))):
                continue
            elif (nurse in chromosome[day-1][2] and nurse in chromosome[day][2]):
                violations += 1

    # Regla 8: "No turno de mañana o tarde inmediatamente después de un turno de noche."
    for nurse in range(Nurses):
        for day in range(1, Days):
            if nurse in chromosome[day-1][2] and (nurse in chromosome[day][0] or nurse in chromosome[day][1]):
                violations += 1

    # Regla 9: "Número igual de turnos para cada enfermera."
    # Calculamos cuántos turnos debería tener cada enfermera (aproximadamente igual)
    total_shifts = Days * Shifts
    expected_shifts_per_nurse = total_shifts // Nurses
    for nurse in range(Nurses):
        assigned_shifts = sum(nurse in chromosome[day][shift] for day in range(Days) for shift in range(Shifts))
        if abs(assigned_shifts - expected_shifts_per_nurse) > 1:
            violations += 1

    # Regla 10: "Mínimo un fin de semana libre al mes."
    # Asumimos que los fines de semana son días 6, 7, 13, 14, 20, 21, 27, 28 en un mes de 30 días.
    weekends = [6, 7, 13, 14, 20, 21, 27, 28]
    for nurse in range(Nurses):
        weekends_free = sum(all(nurse not in chromosome[day][shift] for shift in range(Shifts)) for day in weekends)
        if weekends_free < 1:
            violations += 1

    return violations

def fitnessFunc(chromosome):
    # Contar el número de violaciones de las reglas en este cromosoma
    violations = count_violations(chromosome)
    
    # Calcular el valor de fitness
    # Se usa 1 / (1 + violations) por las siguientes razones:
    # 1. Asegura que el fitness siempre sea positivo (importante para la selección por ruleta)
    # 2. Un cromosoma con 0 violaciones tendrá el máximo fitness de 1
    # 3. A medida que aumentan las violaciones, el fitness se acerca a 0 pero nunca lo alcanza
    # 4. Esta fórmula crea una relación inversa entre violaciones y fitness
    return 1 / (1 + violations)

def roulette(population):
    # Calcular el valor de fitness para cada individuo en la población
    fitness_values = np.array([fitnessFunc(chrom) for chrom in population])
    
    # Normalizar los valores de fitness para obtener probabilidades
    # Cada probabilidad es el fitness del individuo dividido por la suma total de fitness
    probability = fitness_values / sum(fitness_values)
    
    # Calcular la probabilidad acumulada
    # Esto crea una "ruleta" donde los individuos con mayor fitness ocupan secciones más grandes
    accumProb = np.cumsum(probability)
    
    return accumProb

def selection(population, accumProb):
    # Generar un número aleatorio entre 0 y 1
    nr = np.random.uniform()
    
    # Recorrer la lista de probabilidades acumuladas
    for i, p in enumerate(accumProb):
        # Si el número aleatorio es menor que la probabilidad acumulada actual,
        # seleccionar el individuo correspondiente
        if nr < p:
            return population[i]
    
    # Si por alguna razón no se seleccionó ningún individuo (lo cual es muy improbable),
    # retornar el último individuo de la población
    return population[-1]

# Operador de mutación
def mutate(chromosome):
    day = np.random.randint(Days)
    shift = np.random.randint(Shifts)
    nurse1, nurse2 = np.random.choice(2, 2, replace=False)
    chromosome[day][shift][nurse1], chromosome[day][shift][nurse2] = chromosome[day][shift][nurse2], chromosome[day][shift][nurse1]
    return chromosome

# Operador de cruce
def crossover(parent1, parent2):
    point = np.random.randint(1, Days)
    child1 = np.concatenate((parent1[:point], parent2[point:]), axis=0)
    child2 = np.concatenate((parent2[:point], parent1[point:]), axis=0)
    return child1, child2

def geneticAlgorithm(sizeIndividual, sizePopulation, generations, mutation_probability, crossover_probability, elitism_size):
    # elitism_size define que cantidad de individuos elite vamos a pasar
    # El resto son cruce
    population = generateFirstPopulation(sizePopulation, Nurses, Days, Shifts)
    accumulatedP = roulette(population)
    ff = [fitnessFunc(i) for i in population]
    bestIndividual = [population[np.argmax(ff)]]

    for i in range(generations):
        newPopulation = []
        
        # Elitismo: Transferir los mejores individuos directamente a la nueva población
        sorted_population = sorted(population, key=fitnessFunc, reverse=True)
        newPopulation.extend(sorted_population[:elitism_size])
        
        # Resto de la población por selección, cruce y mutación
        while len(newPopulation) < sizePopulation:
            # Selección de padres
            ind1 = selection(population, accumulatedP)
            ind2 = selection(population, accumulatedP)

            # Cruce con probabilidad crossover_probability
            crossProb = np.random.uniform()
            if crossProb <= crossover_probability:
                child1, child2 = crossover(ind1, ind2)
                
                # Mutación en los hijos del cruce
                mProb1 = np.random.uniform()
                mProb2 = np.random.uniform()
                if mProb1 <= mutation_probability:
                    child1 = mutate(child1)
                if mProb2 <= mutation_probability:
                    child2 = mutate(child2)
                    
                if valid(child1):
                    newPopulation.append(child1)
                if valid(child2):
                    newPopulation.append(child2)
            else:
                newPopulation.append(ind1)
                newPopulation.append(ind2)

        # Actualización de la población y de la ruleta para la próxima generación
        accumulatedP = roulette(newPopulation)
        ff = [fitnessFunc(i) for i in newPopulation]
        bestIndividual.append(newPopulation[np.argmax(ff)])
        population = newPopulation

    # Ordenar los mejores individuos según su aptitud
    sortedBest = sorted(bestIndividual, key=fitnessFunc, reverse=True)
    return bestIndividual, sortedBest[0]


In [8]:
# Ejecución del algoritmo

bestList, bestInd = geneticAlgorithm(Nurses, Population_Size, Generations, Mutation_Probability, Crossover_Probability, Elitism_Size)

# Evaluación del mejor individuo
bestInd

array([[[6, 7],
        [9, 3],
        [1, 9]],

       [[3, 3],
        [3, 7],
        [5, 9]],

       [[0, 4],
        [1, 8],
        [6, 5]],

       [[8, 5],
        [3, 4],
        [7, 9]],

       [[2, 1],
        [0, 6],
        [1, 2]],

       [[8, 5],
        [4, 6],
        [7, 5]],

       [[5, 3],
        [3, 2],
        [5, 1]],

       [[3, 7],
        [9, 2],
        [4, 3]],

       [[9, 9],
        [9, 5],
        [4, 3]],

       [[7, 9],
        [9, 1],
        [8, 8]],

       [[8, 5],
        [5, 5],
        [1, 4]],

       [[5, 0],
        [8, 3],
        [6, 4]],

       [[4, 2],
        [8, 1],
        [2, 2]],

       [[7, 8],
        [4, 9],
        [6, 3]],

       [[7, 9],
        [8, 5],
        [3, 9]],

       [[8, 2],
        [5, 1],
        [1, 6]],

       [[6, 6],
        [0, 1],
        [8, 3]],

       [[7, 5],
        [0, 4],
        [1, 6]],

       [[4, 8],
        [9, 5],
        [0, 8]],

       [[2, 3],
        [7, 3],
        [0, 5]],



In [ ]:
#Imprimamos la cantidad de violaciones
count_violations(bestInd)